<h1><b>Ο αλγόριθμος Logistic Regression</b></h1>
<p align="justify">Στην άσκηση αυτή θα μελετήσετε τον αλγόριθμο <i>logistic regression</i>, κατασκευάζοντας ένα σύντομο πρόγραμμα. Στην άσκηση αυτή θα χρησιμοποιήσετε τη βιβλιοθήκη της <i>Python Scikit-Learn</i>. Για τη διευκόλυνσή σας παρέχονται οι δηλώσεις των βιβλιοθηκών που θα χρησιμοποιήσετε καθώς και εντολές με κενά. Συγκεκριμένα, θα κατασκευάσετε έναν <i>ταξινομητή Spam μηνυμάτων SMS</i>. Τα δεδομένα που θα χρησιμοποιήσετε για την εκπαίδευση και την επικύρωση του μοντέλου, καθώς και πληροφορίες για αυτά μπορούν να βρεθούν <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection"><i>εδώ</i></a>.</p>
<p align="justify">Καλείστε να αναπτύξετε ένα πρόγραμμα, το οποίο:</p>
<ul>
<li>Θα φορτώνει τα δεδομένα από το αρχείο <i>.csv</i>.</li>
<li>Θα διαχωρίζει με τυχαίο τρόπο τα δεδομένα που παρέχονται σε δεδομένα εκπαίδευσης <i>(training set)</i> και επικύρωσης <i>(test set)</i>.</li>
<li>Θα εκπαιδεύει το μοντέλο logistic regression.</i>
<li>Θα υπολογίζει την ακρίβειά του πάνω στο <i>test set</i>. Περισσότερες πληροφορίες για τη μέθοδο <i>TfidVectorizer</i>, που περιλαμβάνεται στις δηλώσεις του προγράμματος, μπορούν να βρεθούν <a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html"><i>εδώ</i></a>.</li>
</ul>
<p align="justify">Για ποιους λόγους πιστεύετε ότι δεν είναι κατάλληλη η εφαρμογή της μεθόδου <i>linear regression</i> στο συγκεκριμένο πρόβλημα;</p>

In [19]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [10]:
# load csv file

#filename =  spam.csv
df  = pd.read_csv('spam.csv',encoding='latin-1')
df= df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
df = df.rename(columns = {'v1':'label','v2':'message'})
df.head(5)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [20]:
# vectorize data
vectorizer = TfidfVectorizer("english")

text_feat = df['message'].copy()

def text_process(text):
    
    #https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')] # remove stopwords
    
    return " ".join(text)

text_feat = text_feat.apply(text_process)

features = vectorizer.fit_transform(text_feat)

In [57]:
# split dataset
X_train, X_test, Y_train, Y_test = train_test_split(features, df['label'], test_size=0.2, random_state=111)

#### logistic regression

In [58]:
lrc = LogisticRegression(solver='liblinear', penalty='l1')
lrc.fit(X_train,Y_train)

lrc.score(X_test,Y_test)

0.9515695067264573

#### linear regression

In [70]:
from sklearn import linear_model

#Y_train[Y_train == 'ham'] = 0
#Y_train[Y_train == 'spam'] = 1

#Y_test[Y_test == 'ham'] = 0
#Y_test[Y_test == 'spam'] = 1

lrg = linear_model.SGDRegressor(max_iter=1000, tol=1e-3,learning_rate='optimal',alpha=0.0001)
lrg.fit(X_train,Y_train)

lrg.score(X_test,Y_test)

/home/zerkes/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:1185: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


-292699464.8284186

### Γιατί χρησιμοποιήσαμε logistic και όχι linear regression?

Δε χρησιμοποιύμε linear regression καθώς έχουμε μεγάλους πίνακες και αραιούς με αποτέλεσμα το MSE ERROR να ξεφεύγει πάρα πολύ, γιατί τα gradients γίνονται πολύ μικρά ή πολύ μεγάλα.

Επίσης, σε ένα τέτοιο δυαδικό πρόβλημα το logistic regression εισάγει την έννοια της πιθανότητας. Επομένως, είναι προσαρμόζει ένα μοντέλο που μας δίνει την πιθανότητα αν κάτι είναι ή όχι SPAM.